In [ ]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf; print('Tensorflow version:',tf.__version__)

In [ ]:
print('Available devices:', tf.config.list_physical_devices())

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3*1024)]
    )

In [ ]:
DIRECTORY = 'dataset'
IMAGE_SIZE = (224, 224)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
BATCH_SIZE = 16
COLOR_MODE='grayscale'

dataset = tf.keras.preprocessing.image_dataset_from_directory(
      directory=DIRECTORY
    , image_size=IMAGE_SIZE
    , batch_size=BATCH_SIZE    
    , color_mode=COLOR_MODE
    #, validation_split=0.0
    #, subset="training"
    , seed=123
)

In [ ]:
Labels = [0]*8
Images = {}
for images, labels in dataset:
    for image, label in zip(images, labels):
        label = int(label)
        Labels[label] += 1  
        if ( label in Images ):
            if  ( len(Images[label]) < 20 ):
                Images[label].append(image)
        else:
            Images[label] = []
            Images[label].append(image)
class_names = dataset.class_names
print(' '.join( ['{:<12}'.format(el) for el in class_names] ))
print(' '.join( ['{:<12}'.format(el) for el in Labels] ))

In [ ]:
# for label in Images:
#     images = Images[label]
#     class_name = class_names[label]
#     f, axs = plt.subplots(2, 10, figsize=(25, 4))
#     axs = axs.reshape(-1)
#     f.suptitle(class_name.upper())
#     for i, image in enumerate(images):
#         ax = plt.subplot(2, 10, i + 1)
#         axs[i].imshow(image.numpy().astype("uint8"), cmap='gray')
#         #axs[i].suptitle(class_names[label], y=-0.20)
#         axs[i].axis("off")    

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

dataset = dataset.prefetch(buffer_size=AUTOTUNE)

# Use data augmentation
ZOOM = (0.0, -0.2)
data_augmentation = tf.keras.Sequential(
    [
          tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal")
        , tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.05, fill_mode='nearest')
        , tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=ZOOM, width_factor=ZOOM)
    ]
)

In [ ]:
import pathlib

classNames  = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutrality', 'sadness', 'surprise']
root = 'dataset_augmented'
pathlib.Path(root).mkdir(exist_ok=True)
for class_name in class_names:
  pathlib.Path(f'{root}/{class_name}').mkdir(exist_ok=True)

images_max  = max(Labels)
images_left = [images_max - qty for qty in Labels]

In [ ]:
while any(images_left): 
  for images, labels in dataset:
    if any(images_left):
      for image, label in zip(images, labels):
        if images_left[label]:
          images_left[label] -= 1
          augmented_image = data_augmentation(tf.expand_dims(image, 0))
          formatted_image = augmented_image[0].numpy()
          tf.keras.utils.save_img(f'{root}/{classNames[label]}/augmented_{images_left[label]}.jpg', formatted_image)
          plt.imshow(formatted_image.squeeze(), cmap='gray')
    else:
      break

# import shutil
# shutil.make_archive(f'{root}', 'zip', root)
